# Redes Neuronales 2021 
Integrantes de grupo:
- Crespo, Pilar
- Müller, Malena
- Scala, Tobías
## TP1: Sesgos en el dataset de SNLI

Uno de los datasets más famosos de Natural Language Inference es SNLI. En esta tarea se debe responder, dadas dos frases A y B, si B es implicación de A ("entailment"), B es contradictorio con A ("contradiction") o si lo que enuncia B es neutral respecto de A ("neutral"). Se dice que A es la premisa y B es la hipótesis.

En Gururangan et al., 2018 mostraron que este dataset tiene algunos sesgos, provocados por ejemplo por las heurísticas que tienen los humanos para generar estos pares de frases (A, B). Para ello, desarrollaron un modelo que aún sin observar la premisa A pudiera clasificar el par (A, B) en alguna de las tres clases del dataset.

En este trabajo práctico intentaremos predecir a qué clase pertenece cada una de las hipótesis sin observar la premisa.

Hemos obtenido informacion sobe las metricas empleadas en los siguientes enlaces:

https://sitiobigdata.com/2019/01/19/machine-learning-metrica-clasificacion-parte-3/#

https://www.iartificial.net/precision-recall-f1-accuracy-en-clasificacion/

Cabe mencionar que en el presente programa se está utilizando el dataset descargado en el siguiente link: https://nlp.stanford.edu/projects/snli/

No se está utilizando los archivos hdf5 que se encuentran en Kaggle porque no se encontró forma de extraerles los datos. Además en Kaggle se tuvo el problema de que no se ha podido descargar la librería Keras para poder implemantar el modelo MLP. Debido a esto, se continuó con el TP con la plataforma Visual Studio Code y con el dataset descargado en el link mencionado.

# Importación de datos

In [1]:
import nltk
from nltk import data
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json


nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package wordnet to C:\Users\Tobias
[nltk_data]     Scala\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Tobias
[nltk_data]     Scala\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Tobias
[nltk_data]     Scala\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Importamos el archivo original del dataset para el procesamiento de datos y nos quedamos con las labels y las segundas oraciones de cada caso, ya que son las hipótesis. La clasificación de a qué clase pertenece cada par de oraciones se hace a partir del análisis de la oración que es la hipótesis.

In [2]:
file = pd.read_json('snli_1.0_train.jsonl', lines=True)
fileClass = file['gold_label']
fileLines = file['sentence2']
#print(fileClass)

# Filtrado / procesamiento de datos 

Tenemos dos tipos de data set. En uno realizamos lo siguiente:
- word_tokenize: Separamos la oración en strings.
- isalpha: Se eliminan los números.
- lemmatize: Se pasa todo a singular y se generaliza el género.
- stem: Se pasan todos los verbos a infinitivo y se deja todo en minúscula.

Y el otro dataset lo obtenemos de la misma forma que el recién mencionado, pero agregándole también la eliminación de stopwords (palabras comunes).

Si bien inicialmente probamos eliminando las stopwords, determinamos que convenía no hacerlo ya que hay palabras comunes como "no", que serían eliminadas, pero en nuestro caso son necesarias estas palabras para determinar que una hipótesis corresponde a una "contradicción", por ejemplo. Para esto, nos basamos en el paper "Annotation Artifacts in Natural Language Inference Data".

In [ ]:
linesFilt = []
for i in range(len(fileLines)):
    if (i % 1000 == 0):
        print(i)
    tok=word_tokenize(fileLines[i]) #Separa la oración en strings.
    alpha=[x for x in tok if x.isalpha()] #Saca palabras con números.
    lem=[lemmatizer.lemmatize(x,pos='v') for x in alpha] #Pasa de plural a singular y generaliza el género.
    #stop=[x for x in lem if x not in stopwords.words('english')] #Saca las palabras comunes.
    stem=[stemmer.stem(x) for x in lem] #Verbos a infinitivo y pasa todo a minuscula.
    linesFilt.append(" ".join(stem))

Guardamos en un json las hipotesis ya procesadas.

In [ ]:
with open('train_processed_.jsonl', 'w') as file:
    for i in range(len(fileClass)):
        data2jsonl = {'gold_label': fileClass[i], 'sentence2': linesFilt[i]}
        json.dump(data2jsonl, file)
        file.write('\n')

Repetimos lo anterior para los datasets de validación y de test.

In [ ]:
# ---- Validation file:
file = pd.read_json('snli_1.0_dev.jsonl', lines=True)
fileClass = file['gold_label']
fileLines = file['sentence2']
print(fileClass)

linesFilt = []
for i in range(len(fileLines)):
    if (i % 1000 == 0):
        print(i)
    tok=word_tokenize(fileLines[i]) #Separa la oración en strings.
    alpha=[x for x in tok if x.isalpha()] #Saca palabras con números.
    lem=[lemmatizer.lemmatize(x,pos='v') for x in alpha] #Pasa de plural a singular y generaliza el género.
    #stop=[x for x in lem if x not in stopwords.words('english')] #Saca las palabras comunes.
    stem=[stemmer.stem(x) for x in lem] #Verbos a infinitivo y pasa todo a minuscula.
    linesFilt.append(" ".join(stem))

with open('val_processed_.jsonl', 'w') as file:
    for i in range(len(fileClass)):
        data2jsonl = {'gold_label': fileClass[i], 'sentence2': linesFilt[i]}
        json.dump(data2jsonl, file)
        file.write('\n')

# ---- Test file:
file = pd.read_json('snli_1.0_test.jsonl', lines=True)
fileClass = file['gold_label']
fileLines = file['sentence2']
print(fileClass)

linesFilt = []
for i in range(len(fileLines)):
    if (i % 1000 == 0):
        print(i)
    tok=word_tokenize(fileLines[i]) #Separa la oración en strings.
    alpha=[x for x in tok if x.isalpha()] #Saca palabras con números.
    lem=[lemmatizer.lemmatize(x,pos='v') for x in alpha] #Pasa de plural a singular y generaliza el género.
    #stop=[x for x in lem if x not in stopwords.words('english')] #Saca las palabras comunes.
    stem=[stemmer.stem(x) for x in lem] #Verbos a infinitivo y pasa todo a minuscula.
    linesFilt.append(" ".join(stem))

with open('test_processed_.jsonl', 'w') as file:
    for i in range(len(fileClass)):
        data2jsonl = {'gold_label': fileClass[i], 'sentence2': linesFilt[i]}
        json.dump(data2jsonl, file)
        file.write('\n')

# Entrenamiento de la red con Naive Bayes

Levantamos los datos ya procesados.

In [1]:
import numpy as np
import pandas as pd #Implementación de clasificador bayesiano.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

trainFile = pd.read_json('train_processed_.jsonl', lines=True)
trainClass = trainFile['gold_label'].tolist()
trainLines = trainFile['sentence2'].tolist()

valFile = pd.read_json('val_processed_.jsonl', lines=True)
valClass = valFile['gold_label'].tolist()
valLines = valFile['sentence2'].tolist()

testFile = pd.read_json('test_processed_.jsonl', lines=True)
testClass = testFile['gold_label'].tolist()
testLines = testFile['sentence2'].tolist()

Obtenemos las matrices dispersas que contienen la frecuencia (cantidad de ocurrencia) de cada palabra del vocabulario.

In [2]:
# countVect = CountVectorizer(max_df=0.8,min_df=10, ngram_range=(1,2)) #Recibe todos los artículos y arma los vectores de cuenta. Check "ngram_range"
tfidfVect = TfidfVectorizer(max_df=0.8,min_df=5, ngram_range=(1,2))#, max_features=1000) #As tf–idf is very often used for text features
#tfidfVect = TfidfVectorizer(max_df=0.25, min_df=1, ngram_range=(1,5))#, max_features=1000) #As tf–idf is very often used for text features

# trainData = countVect.fit_transform(trainLines) #Learn a vocabulary dictionary of all tokens in the raw documents and return document-term matrix.
# print(trainData.shape) #En este sparse matrix se muestran las ocurrencias de cada palabra en cada linea.
trainData = tfidfVect.fit_transform(trainLines) #Learn vocabulary and idf from training set, return document-term matrix.
print(trainData.shape) #En este sparse matrix se muestran las ocurrencias de cada palabra en cada linea. Con tfidf las ocurrencias de las palabras no se representan con números enteros.

# valData = countVect.transform(valLines) #Transform documents to document-term matrix. Extract token counts out of raw text documents using the vocabulary fitted with fit.
# print(valData.shape)
valData = tfidfVect.transform(valLines) #Transform documents to document-term matrix. Extract token counts out of raw text documents using the vocabulary fitted with fit.
print(valData.shape)

# valData = countVect.transform(valLines) #Transform documents to document-term matrix. Extract token counts out of raw text documents using the vocabulary fitted with fit.
# print(valData.shape)
testData = tfidfVect.transform(testLines) #Transform documents to document-term matrix. Extract token counts out of raw text documents using the vocabulary fitted with fit.
print(testData.shape)

(550152, 56323)
(10000, 56323)
(10000, 56323)


Entrenamos el modelo utilizando Naive Bayes y obtenemos el score con los datos de train y el score con los nuevos datos (validation).

In [3]:
multiNB = MultinomialNB(alpha=0.65)
multiNB.fit(trainData, trainClass)

print(multiNB.score(trainData, trainClass))
print(multiNB.score(valData, valClass))
print(multiNB.score(testData, testClass))

testPred = multiNB.predict(testData)
# df_test = pd.DataFrame(data=testPred, columns=["pred_labels"],) #Armo el submission.csv
# #df_test.head()
# df_test.index.names = ["pairID"]
# df_test.to_csv("submission.csv")

0.6657032965435007
0.6412
0.6416


Comprobamos la efectividad del modelo prediciendo los nuevos datos.






## Metricas secundarias: Precision

La metrica precision es la relacion $\frac{tp}{tp + fp}$ , siendo tp el numero de correctos positivos y fp el numero de falsos positivos. Es la abilidad del clasificador de no clasificar como positivo aquello que es negativo. Es decir, es la cantidad de elementos identificados correctamente como positivos, respecto al total de elementos identificados como positivos. El major valor es un 1 y el peor valor es un 0.
Es el número de elementos identificados correctamente como positivo de un total de elementos identificados como positivos.







In [4]:
from sklearn.metrics import precision_score
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(testClass)
testClass = enc.transform(testClass)
testClass = np_utils.to_categorical(testClass-1) #Convert integers to dummy variables (i.e. one hot encoded).
enc.fit(testPred)
testPred = enc.transform(testPred)
testPred = np_utils.to_categorical(testPred) #Convert integers to dummy variables (i.e. one hot encoded).

print(precision_score(testClass, testPred, average=None))
# print(precision_score(valClass, valPred, average='micro'))
# print(precision_score(valClass, valPred, average='macro'))
# print(precision_score(valClass, valPred, average='weighted'))

[0.63434903 0.64125178 0.66656366]


## Métrica secundaria: ROC-AUC 

Este método se basa en la curva ROC que se vincula con las características de funcionamiento del receptor y el área bajo esta curva. ROC es una curva que permite ver que tan efectiva es la clasificación del modelo. Se determina un umbral, de forma que los valores que resulten positivos por encima del umbral son verdaderos positivos, los valores negativos por encima del umbral son falsos positivos, los valores negativos debajo del umbral son verdaderos negativos y los valores positivos debajo del umbral son falsos negativos.

La sensibilidad  es la proporción de verdaderos positivos sobre el total de resultados positivos.
La especificidad es la proporción de verdaderos negativos sobre el total de resultados negativos.
Si tomamos un valor de umbral menor, se obtienen más resultados negativos, de forma que aumenta la sensibilidad y disminuye la especificidad. Por el contrario, si tomamos un valor de umbral mayor, se obtienen más resultados negativos y entonces aumenta la especificidad y disminuye la sensibilidad.
La curva ROC es la sensibilidad en función de 1-especificidad. La sensibilidad da la tasa de verdaderos positivos mientras que 1-especificidad da la tasa de falso positivo. Es por esto que al aumentar la sensibilidad, 1-especificidad también aumenta. 
AUC, por otro lado, es el área bajo la curva ROC. 

Ventajas de la métrica ROC-AUC:
* No varía respecto a la escala. Mide la efectividad de las predicciones, en lugar de sus valores absolutos.
* No varía respecto al umbral de clasificación. Mide la calidad de las predicciones del modelo, sin considerar el umbral de clasificación.

In [5]:
from sklearn.metrics import roc_auc_score

#clf = LogisticRegression(solver="liblinear").fit(X, )
print(roc_auc_score(testClass, testPred, multi_class='macro'))

0.7353541918174505


## Metricas secundarias: F1-Score

Combina las medidas de precision y recall. Esto es práctico porque hace más fácil el poder comparar el rendimiento combinado de la precisión y la exhaustividad entre varias soluciones.

$F1 = 2 \cdot \frac{presicion \cdot recall}{presicion + recall}$


In [6]:
from sklearn.metrics import f1_score

print(f1_score(testClass, testPred, average='micro'))
print(f1_score(testClass, testPred, average='macro'))
print(f1_score(testClass, testPred, average='weighted'))


0.6472
0.6470167470802781
0.6471769419857526


## Metricas secundarias: Recall


Es el número de elementos identificados correctamente como positivos del total de positivos verdaderos.

$recall = \frac{tp}{tp+fn}$


In [7]:
from sklearn.metrics import recall_score

print(recall_score(testClass, testPred, average='micro'))
print(recall_score(testClass, testPred, average='macro'))
print(recall_score(testClass, testPred, average='weighted'))

0.6472
0.6470955503129107
0.6472


# Entrenamiento de la red con MLP

Truncamos el vocabulario para no tener error por falta de memoria.

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.decomposition import TruncatedSVD

trunSVD = TruncatedSVD(n_components=1000)
trainData = trunSVD.fit_transform(trainData)
testData = trunSVD.transform(testData)
print(trainData.shape)

# enc = LabelEncoder()
enc.fit(trainClass)
trainClass = enc.transform(trainClass)
trainClass = np_utils.to_categorical(trainClass-1) #Convert integers to dummy variables (i.e. one hot encoded).
# valClass = enc.transform(valClass)
# valClass = np_utils.to_categorical(valClass-1) #Convert integers to dummy variables (i.e. one hot encoded).

(550152, 1000)


## Métrica primaria: Accuracy

Accuracy es el porcentaje de elementos clasificados correctamente, por lo tanto, es un valor entre 0 y 1. Cuanto mayor es este valor, significa que mejor funciona la clasificación.

* Ventaja: es la métrica más intuitiva y simple de usar.

* Desventajas:
La predicción a partir de la clase más frecuente puede no ser buena si las clases a las que pertenecen los datos no están balanceadas.

Generamos la estructura de la red neuronal con la siguiente función.

In [9]:
def neuralNetwork():
    Nwords = trainData.shape[1] #cantidad de datos (palabras del vocabulario). ###
    model = Sequential()
    model.add(Dense(200, input_shape=(Nwords,), activation='relu'))
    model.add(Dense(trainClass.shape[1], activation='softmax')) #La salida de la función softmax puede ser utilizada para representar una distribución categórica. 
                                                #Es empleada en varios métodos de clasificación multiclase tales como Regresión Logística Multinomial.
                                                #La función softmax es utilizada como capa final de los clasificadores basados en redes neuronales.
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Entrenamos la red neuronal y obtenemos el accuracy del mismo.

In [10]:
#Cross validation.
# estClass = KerasClassifier(build_fn=neuralNetwork, epochs=50, batch_size=256, verbose=0) #Estimator.
# Kfold = KFold(n_splits=10, shuffle=True)
# scores = cross_val_score(estClass, trainData_.todense(), encClass_, cv=Kfold)
# print("Score: %.2f%% (%.2f%%)" % (scores.mean()*100, scores.std()*100))
#Hold out
model = neuralNetwork()
model.fit(trainData, trainClass, epochs=10, batch_size=256, verbose=1)
loss, acc = model.evaluate(testData, testClass)
print('Test Accuracy: %f' % (acc))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               200200    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 603       
Total params: 200,803
Trainable params: 200,803
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2150/2150 [==============================] - 13s 6ms/step - loss: 0.8676 - accuracy: 0.6030
Epoch 2/10
2150/2150 [==============================] - 12s 6ms/step - loss: 0.8344 - accuracy: 0.6215
Epoch 3/10
2150/2150 [==============================] - 12s 6ms/step - loss: 0.8131 - accuracy: 0.6354
Epoch 4/10
2150/2150 [==============================] - 12s 6ms/step - loss: 0.7946 - accuracy: 0.6465
Epoch 5/10
2150/2150 [==============================] - 12s 5ms/step - loss: 0.7787 - accuracy: 0.65

Comprobamos la eficiencia del modelo con los nuevos datos (validation).
## Métrica secundaria: Precisión.

In [11]:
testPred = model.predict(testData, verbose = 1)
testPred = testPred.round()

print(precision_score(testClass, testPred, average=None))
# print(precision_score(valClass_, valPred, average='micro'))
# print(precision_score(valClass_, valPred, average='macro'))
# print(precision_score(valClass_, valPred, average='weighted'))

313/313 [==============================] - 0s 2ms/step
[0.71178274 0.70719424 0.67961499]


## Métrica secundaria: ROC-AUC

In [12]:
from sklearn.metrics import roc_auc_score

#clf = LogisticRegression(solver="liblinear").fit(X, )
print(roc_auc_score(testClass, testPred, multi_class='macro'))

0.7178211062117722


# Comparacion entre las metricas empleadas

Accuracy es la métrica más intuitiva y simple de usar, pero tiene como desventaja que la predicción a partir de la clase más frecuente puede no ser buena si las clases a las que pertenecen los datos no están balanceadas. Puede hacer creer que el modelo funciona mejor de como realmente funciona.

Las otras metricas empleadas en este trabajo practico son mas representativas y tienen en cuenta el balance entre clases. 

Presicion da la calidad de la prediccion ya que indica, de lo que fue clasificado como que pertenece a una dada clase, el porcentaje que realmente pertenece a esa clase.

Recall indica, del total de datos pertenecientes a una dada clase, que porcentaje pudo ser correctamente clasificado.

F1 es una metrica que combina las metricas presicion y recall.

Una ventaja de la métrica ROC-AUC es que no varía respecto a la escala. Mide la efectividad de las predicciones, en lugar de sus valores absolutos. Y ademas, no varía respecto al umbral de clasificación. Mide la calidad de las predicciones del modelo, sin considerar el umbral de clasificación.

Por lo tanto, es importante tener en cuenta que es lo que se quiere rescatar del resultado de la clasificacion del modelo para determinar la metrica a usar para evaular dicho modelo. No conviene usar accuracy, a no ser que la cantidad de datos que pertenecen a cada clase este balanceada.






...